In [5]:
import pickle
import numpy as np
from transformers import *
from image_helper import * 

with open('./data/southwest_images_new_train.pkl', 'rb') as train_f:
    saved_southwest_dataset_train = pickle.load(train_f)
    print(saved_southwest_dataset_train)
with open('./data/southwest_images_new_test.pkl', 'rb') as test_f:
    saved_southwest_dataset_test = pickle.load(test_f)

print('shape of edge case train data (southwest airplane dataset train)',saved_southwest_dataset_train.shape)
print('shape of edge case test data (southwest airplane dataset test)',saved_southwest_dataset_test.shape)

# np.ones((x,y), dype=int) 建立一个[x,y]维的int型数组，且值为1，再*9
sampled_targets_array_train = 9 * np.ones((saved_southwest_dataset_train.shape[0],), dtype =int)
print(sampled_targets_array_train)
sampled_targets_array_test = 9 * np.ones((saved_southwest_dataset_test.shape[0],), dtype =int)
print(np.max(saved_southwest_dataset_train))

transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
        ])

trainset = Customize_Dataset(X=saved_southwest_dataset_train, Y=sampled_targets_array_train, transform=transform)
poisoned_train_loader = DataLoader(dataset = trainset, batch_size = 64, shuffle = True, num_workers=1)
print(trainset)

testset = Customize_Dataset(X=saved_southwest_dataset_test, Y=sampled_targets_array_test, transform=transform)
poisoned_test_loader = DataLoader(dataset = testset, batch_size = 64, shuffle = True, num_workers=1)


[[[[217 229 241]
   [222 233 244]
   [227 236 245]
   ...
   [212 225 239]
   [204 220 236]
   [196 214 231]]

  [[223 234 243]
   [228 236 246]
   [231 238 247]
   ...
   [216 228 240]
   [210 223 237]
   [202 218 233]]

  [[227 236 244]
   [230 238 247]
   [234 240 248]
   ...
   [219 230 241]
   [214 226 238]
   [208 221 233]]

  ...

  [[213 205 200]
   [216 208 203]
   [217 209 203]
   ...
   [206 195 190]
   [196 187 181]
   [185 175 170]]

  [[201 192 187]
   [206 196 191]
   [208 198 193]
   ...
   [205 197 191]
   [197 188 183]
   [186 177 172]]

  [[197 187 182]
   [201 190 184]
   [204 193 187]
   ...
   [205 197 191]
   [197 188 183]
   [184 175 170]]]


 [[[196 214 231]
   [202 218 233]
   [208 221 233]
   ...
   [185 175 170]
   [186 177 172]
   [184 175 170]]

  [[204 220 236]
   [210 223 237]
   [214 226 238]
   ...
   [196 187 181]
   [197 188 183]
   [197 188 183]]

  [[212 225 239]
   [216 228 240]
   [219 230 241]
   ...
   [206 195 190]
   [205 197 191]
   [205 197

In [37]:
import torchvision
from torchvision import transforms


transform_train = transforms.Compose([
            transforms.RandomCrop(32, padding=4),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
        ])

transform_test = transforms.Compose([
                    transforms.ToTensor(),
                    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
                ])

trainset = torchvision.datasets.CIFAR10(root = './data',
                                        train = True,
                                        download = False,
                                        transform=None)

In [66]:
import PIL as plt
from PIL import Image
import numpy as np

print(trainset[49999][0])
x = np.array(trainset[49999][0])
print(x[:,:,0])
print(x)

<PIL.Image.Image image mode=RGB size=32x32 at 0x2723EDE2490>
[[229 236 234 ... 217 221 222]
 [222 239 233 ... 223 227 210]
 [213 234 231 ... 220 220 202]
 ...
 [150 140 132 ... 224 230 241]
 [137 130 125 ... 181 202 212]
 [122 118 120 ... 179 164 163]]
[[[229 229 239]
  [236 237 247]
  [234 236 247]
  ...
  [217 219 233]
  [221 223 234]
  [222 223 233]]

 [[222 221 229]
  [239 239 249]
  [233 234 246]
  ...
  [223 223 236]
  [227 228 238]
  [210 211 220]]

 [[213 206 211]
  [234 232 239]
  [231 233 244]
  ...
  [220 220 232]
  [220 219 232]
  [202 203 215]]

 ...

 [[150 143 135]
  [140 135 127]
  [132 127 120]
  ...
  [224 222 218]
  [230 228 225]
  [241 241 238]]

 [[137 132 126]
  [130 127 120]
  [125 121 115]
  ...
  [181 180 178]
  [202 201 198]
  [212 211 207]]

 [[122 119 114]
  [118 116 110]
  [120 116 111]
  ...
  [179 177 173]
  [164 164 162]
  [163 163 161]]]


In [1]:
from typing import Text
from yaml import tokens
import torch
from torch.autograd import Variable
from torch.utils.data import DataLoader, TensorDataset
from torchvision.datasets import MNIST, EMNIST

from helper import Helper
import random
from utils.text_load import Dictionary
from models.word_model import RNNModel
from models.resnet import ResNet18
from models.lenet import LeNet
from models.edge_case_cnn import Net
from models.resnet9 import ResNet9
from utils.text_load import *
import numpy as np
import copy
from torch.utils.data import Dataset, DataLoader

import os
from torchvision import datasets, transforms
from collections import defaultdict
from torch.utils.data import DataLoader, random_split, TensorDataset
import pickle
from image_helper import Customize_Dataset
from image_helper import *
from image_helper import get_poison_cifar10_train_label


transform_test = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
        ])
poison_cifar10_train = get_poison_cifar10()
test_dataset = datasets.CIFAR10('X:\Directory\code\Durable-Federated-Learning-Backdoor\FL_Backdoor_CV\data', train=False, transform=transform_test)
sampled_targets_array_test = get_poison_cifar10_train_label()


poison_testset = Customize_Dataset(X=poison_cifar10_train, Y=sampled_targets_array_test, transform=transform_test)
cifar_poison_classes_ind = []
label_list = []
print(poison_testset[0])
for ind, x in enumerate(poison_testset):
    imge, label = x
    label_list.append(label)
    if label == 5:
        cifar_poison_classes_ind.append(ind)


print(cifar_poison_classes_ind)

d:\Anaconda3\envs\neurotoxin\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


(tensor([[[-1.1884, -1.0721,  0.3042,  ...,  0.0716,  0.0134, -0.0641],
         [-1.7894, -0.3161, -0.2580,  ..., -0.2773, -0.0060, -1.1109],
         [-0.4906,  0.1104,  0.1491,  ..., -0.9752,  0.1491, -0.0253],
         ...,
         [-0.9364, -1.5374, -1.2854,  ..., -1.0915, -1.5180, -0.5681],
         [-0.7813, -1.6537, -1.3241,  ..., -1.4598, -1.1497, -0.8201],
         [-0.5487, -1.9251, -1.6924,  ..., -1.2660, -1.5180,  0.3624]],

        [[-1.5529, -0.4122,  0.4334,  ...,  0.2171,  0.2171, -0.0582],
         [-1.7299, -0.0976, -0.3139,  ...,  0.1384, -0.3729, -1.8282],
         [-0.1959,  0.2564,  0.1974,  ..., -0.2942,  0.0991, -0.0779],
         ...,
         [-1.4939, -1.1596, -1.2579,  ..., -1.2972, -1.4742, -0.2746],
         [-1.8086, -1.4152, -1.2579,  ..., -1.4546, -1.4152, -2.2806],
         [-1.7299, -1.5922, -1.3169,  ..., -1.1596, -1.2186, -0.8056]],

        [[-1.2069,  0.2174,  0.7052,  ...,  0.3540,  0.2759, -0.0558],
         [-0.4655,  0.0223, -0.1143,  ...,  

50000
